In [13]:

import numpy as np
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from hydra import initialize, compose
from pyHalo.preset_models import CDM

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml', overrides=['machine=local'])  # 

array_dir, data_dir, figure_dir, pickle_dir, repo_dir = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.utils import util

In [14]:
lens = SampleStrongLens()

print(f'Lens redshift: {lens.z_lens}, source redshift: {lens.z_source}')

log_m_host = np.log10(lens.get_main_halo_mass())

Lens redshift: 0.643971, source redshift: 1.627633


In [15]:
pipeline_params = util.hydra_to_dict(config.pipeline)
subhalo_cone = pipeline_params['subhalo_cone']
los_normalization = pipeline_params['los_normalization']

# circumvent bug with pyhalo, sometimes fails when redshifts have more than 2 decimal places
z_lens_rounded = round(lens.z_lens, 2)
z_source_rounded = round(lens.z_source, 2)

# generate subhalos
cdm_realization = CDM(z_lens_rounded, z_source_rounded, log_m_host=log_m_host, r_tidal=0.25,
                      cone_opening_angle_arcsec=subhalo_cone,
                      LOS_normalization=los_normalization)

In [16]:
# add subhalos
lens.add_subhalos(cdm_realization, suppress_output=False)

Main halo mass (10^12 solar masses): 0.578 / Einstein radius: 0.975908
Total subhalo mass: 24211567019.493935
Subhalos are 4.2% of total mass
Adjusted main halo mass: 0.554 / Adjusted Einstein radius: 0.9552553289219334
